In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader,TensorDataset
import torch.utils.data as data
import numpy as np
import scipy.io as sio
import math
import argparse
import random
import os

In [2]:
BATCH_SIZE = 32
EPISODE = 1
TEST_EPISODE = 1000
LEARNING_RATE =1e-3
GPU = 0
Margin = 1

In [3]:
print("init dataset")
##################################参数##################################################################
dataroot = '../data'
dataset = 'AwA1_data'
image_embedding = 'res101'               #ResNet101层
class_embedding = 'original_att'         #属性表达 85-d
#######################################读取视觉特征###################################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + image_embedding + ".mat")  #scipy loadmat
 
feature = matcontent['features'].T         #转置 30478x2048 每一行是一个完整的样本

label = matcontent['labels'].astype(int).squeeze() - 1   #matlab begin 1 ,numpy begin 0
########################################读取属性特征###########################################################

matcontent = sio.loadmat(dataroot + "/" + dataset + "/" + class_embedding + "_splits.mat")

    
# numpy array index starts from 0, matlab starts from 1
trainval_loc = matcontent['trainval_loc'].squeeze() - 1    #squeeze()去掉维度中的1 AxBx1 --->AxB

test_seen_loc = matcontent['test_seen_loc'].squeeze() - 1
test_unseen_loc = matcontent['test_unseen_loc'].squeeze() - 1

attribute = matcontent['att'].T    #转置 50x85 每行是整个属性向量

x = feature[trainval_loc]                      # train_features trainval里面是图片的编号 begin with 0 ，19832个
train_label = label[trainval_loc].astype(int)  # train_label  int类型没变 每个图片的lable 19832个
train_id = np.unique(train_label)

att = attribute[train_label]                   # train attributes 每个图片的属性 19832个
    
x_test = feature[test_unseen_loc]                   # test_feature 5685个
test_label = label[test_unseen_loc].astype(int)     # test_label   5685个

x_test_seen = feature[test_seen_loc]                #test_seen_feature 4958个
test_label_seen = label[test_seen_loc].astype(int)  # test_seen_label  4958个
    
test_id = np.unique(test_label)                     # test_id  10个类 ，unique去重
att_pro = attribute[test_id]                        # test_attribute 每一类的属性向量 10x85

# train set
train_features = torch.from_numpy(x)   #np-->tensor
#train_fearures_negative = torch.from_numpy(x_negative)

sample_attributes=[]
train_label = torch.from_numpy(train_label).unsqueeze(1) #每张图片的属性转化 ，unsqueeze(1)就是插入到第一维度 AxB维-->Ax1xB
#train_negative_label = torch.from_numpy(x_negative_label).unsqueeze(1)
# attributes
all_attributes = np.array(attribute)  #所有50类属性转变为numpy数组???属性向量仍然用的numpy类型 没有转化为pytorch
#print(all_attributes)
    
#print('-'*50)
attributes = torch.from_numpy(attribute) 
#print(attribute)
# test set

test_features = torch.from_numpy(x_test)
#print(test_features.shape)

test_label = torch.from_numpy(test_label).unsqueeze(1)
#print(test_label.shape)

testclasses_id = np.array(test_id)
#print(testclasses_id.shape)

test_attributes = torch.from_numpy(att_pro).float()
#print(test_attributes.shape)

test_seen_features = torch.from_numpy(x_test_seen)
#print(test_seen_features.shape)

test_seen_label = torch.from_numpy(test_label_seen)

train_data = TensorDataset( train_label, train_features )
#train_data = TensorDataset(train_label, train_features, train_fearures_negative)

#################here need new code to make triplet data#####################
print('-'*100)

init dataset
----------------------------------------------------------------------------------------------------


In [4]:
class AttributeNetwork(nn.Module):
    
    def __init__(self, input_size, hidden1_size):
        super(AttributeNetwork, self).__init__()
        
        self.fc1 = nn.Linear(input_size, hidden1_size)   #FC1 network
        

    def forward(self, x):

        x = F.relu(self.fc1(x))      #activate
        
        return x

In [5]:
# init network
print("init networks")

triplet_network = AttributeNetwork(85,2048)  #85d属性 1024隐藏层 2048输出 85d到2048d
triplet_network.cuda(GPU)   

triplet_network_optim = torch.optim.SGD(triplet_network.parameters(), lr=LEARNING_RATE)
triplet_network_scheduler = StepLR(triplet_network_optim , step_size=100000 , gamma=0.5)
#
print('-'*100)

init networks
----------------------------------------------------------------------------------------------------


In [6]:
def _pairwise_distance(x, squared=False, eps=1e-16):
    # Compute the 2D matrix of distances between all the embeddings.

    cor_mat = torch.matmul(x, x.t())
    norm_mat = cor_mat.diag()
    distances = norm_mat.unsqueeze(1) - 2 * cor_mat + norm_mat.unsqueeze(0)
    distances = F.relu(distances)

    if not squared:
        mask = torch.eq(distances, 0.0).float()
        distances = distances + mask * eps
        distances = torch.sqrt(distances)
        distances = distances * (1.0 - mask)

    return distances


In [22]:
print("training...")
last_accuracy = 0.0
for episode in range(EPISODE):
    #attribute_network.train()
    triplet_network_scheduler.step(episode)

    train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

    batch_labels, batch_features = train_loader.__iter__().next()
    batch_id = np.unique(batch_labels)
    print(batch_id)
    print(batch_id.size)
    #batch_attributes = torch.Tensor([all_attributes[i] for i in batch_labels.numpy()]).squeeze(1)
    batch_attributes = torch.Tensor([all_attributes[i] for i in batch_id]).squeeze(1)
    
    batch_features_ext = torch.from_numpy(batch_features.numpy().repeat(batch_id.size, 0))
    batch_attributes_ext = batch_attributes.repeat(BATCH_SIZE, 1)
    
    #print(batch_features_ext)
    #print(batch_attributes_ext)
    
    batch_features_ext = Variable(batch_features_ext).cuda(GPU).float()  # 32*2048
    batch_attributes_ext = Variable(batch_attributes_ext).cuda(GPU)

    relations = F.pairwise_distance(batch_features_ext,triplet_network(batch_attributes_ext),2).view(-1,batch_id.size)
    print(relations.size())
    print(relations)
    

training...
[ 0  1  2  5  9 13 14 15 17 19 31 32 34 35 37 38 39 44 45 47]
20
torch.Size([32, 20])
tensor([[ 502.2079,  638.7623,  613.8737,  532.3043,  597.7808,  487.0503,
          674.7408,  621.7243,  613.1880,  593.6463,  635.9902,  541.7582,
          461.9293,  480.2818,  666.5804,  654.9218,  584.5743,  644.8117,
          509.3112,  564.7006],
        [ 506.8288,  640.9348,  617.6264,  535.1071,  600.7701,  489.9683,
          677.8809,  626.0500,  617.0767,  596.5905,  638.6587,  544.6009,
          464.1793,  483.6032,  670.5908,  658.0546,  589.1163,  648.5335,
          512.7575,  567.7009],
        [ 505.8647,  641.6304,  616.9704,  534.5817,  600.5539,  490.0431,
          678.0186,  625.7833,  616.4929,  595.6705,  639.1935,  543.9169,
          464.8533,  483.5580,  669.4778,  656.7972,  588.5330,  648.8541,
          512.2724,  566.7529],
        [ 507.6316,  643.0777,  618.7258,  537.8409,  603.1451,  491.0892,
          679.2961,  627.0961,  618.6427,  598.0492,  64